In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data_Washington Fatal Crash Survey.csv", low_memory=False)
zips = pd.read_csv("US%20Zip%20Codes%20from%202013%20Government%20Data.txt")

In [9]:
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS



#'methdrug', 'drugsts', drugtst1-12, drugres1-12, 'alcres', 'alcsts', 'alctst'
#ejectpath alcmeth
#investjur repjur

We will drop all columns that are more than 90% NANs because there are not enough values for them to be significant. We'll also drop columns that are not relevant to the problem statement: ['hosp_tm','arr_tm','not_tm','investjur','repjur','lab','methdrug','alcmeth','drugsts','drugtst1']

## Notes
- Before COVID -- seattle has lots of commuters every day
- Since COVID -- people moving farther away from city centers
- Relevant for rural areas + out of staters as well


- how to prevent fatal crashes? 
- Are the people causing crashes the ones that live there? 
- Is it the zipcode one over?or nearby? --> use haversine distance!
- Are there certain zipcodes that PRODUCE high risk drivers? (age, intox, race)
- Who's involved? Cars/bikes/trucks/people --> we look at the **driver**
- also matters: average speed limit of area


- are we allowed to pull external data such as economic status/median income/median home price
- reverse geocoding ! 
- hmmmmm could we find the home address of the drivers lol 


- crash zip codes (crash level)--> maybe 3+ drivers from diff places involved in the crash... but who's at fault? that's what matters
- census data 
- rubric criteria: add additional data -- income data 
- there's a freeway exit in WASH that's notorious for crashes because high speed lim + narrow curve 
- 5 years of data 2017-2021 -- west coast hit by covid first 2019-2020 investigate changes in commuting patterns, increase in substance abuse, people driving faster because less people on the road 

### Derive zip code for crashes 

In [10]:
# long : x
# lat : y

# driver zip code : dzip 
df.rename(columns={'x': 'lon', 'y': 'lat'}, inplace=True)

In [12]:
gis = GIS(api_key="AAPK206d8c337d5a416ca8a7824330c0ca7bkQWmIxSD8K5_qxX1ly5btaWLcl36cVX_1iDwCQxl_IgY56Zh6TN-Usx26i4oM77l")
# exposed for now

def get_zip(df,lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]),"y":float(df[lat_field]), "spatialReference":{"wkid":4326}})))
    return location['address']['Postal']

df['crashzip'] = df.apply(get_zip, axis=1, lat_field='lat', lon_field='lon')

In [83]:
# update missing crash zip values in df 
# case 78 03/18/2020 in county 45(Pacific) --> crash lat long improperly recorded  78.304936,  -778.304936 impossible! city = unincorporated
# FOREST SERVICES RD 2300 is not an actual road... should we drop the row?
# case 349 09/12/2021 in county 5 --> zip: 99352 could not be identified by arcGIS identified using haversine distance
# case 638 09/05/2021 in county 19(Ferry) --> lat long recorded as 78.304936,  -778.304936 impossible! city = unincorporated
# Ferry county is on a reservation in the middle of nowhere next to a natl forest far northeast of WA 
# crash occurred on BRIDGE CREEK RD in Ferry County --> brute force zip: 99138

df.loc[3690,"crashzip"] = 99352
df.loc[4128,"crashzip"] = 99138

In [84]:
# saved because computationally expensive 
# don't want to call API every time we run the code 
# DONT RUN THAT PART 

df.to_csv('crashdata_updated.csv') # import THIS data set

In [85]:
# start from here

data = pd.read_csv("crashdata_updated.csv", low_memory=False) # some cols have mixed dtypes...
data["crash_dt"]= pd.to_datetime(data["crash_dt"])

In [248]:
# df = df.loc[:, df.isnull().mean() < .9]

In [98]:
# Among drivers involved in fatal crashes, what proportion are involved in crashes in communities where they live?

# How do we want to define "community"? Zip code should be fine... 

data['dzip'] = data['dzip'].astype(str)
# get rid of the trailing zeros
data['dzip'] = data['dzip'].replace(r'\.0$', '', regex=True)

In [110]:
print("The proportion of drivers involved in fatal crashes where they don't live = {}".format(len(data.query('dzip != crashzip'))/len(data)))

The proportion of drivers involved in fatal crashes where they don't live = 0.7628267182962246


In [109]:
print("The proportion of drivers involved in fatal crashes where they live = {}".format(len(data.query('dzip == crashzip'))/len(data)))

The proportion of drivers involved in fatal crashes where they live = 0.2371732817037754
